# FP tree

In [1]:
!pip install mlxtend --upgrade

     |████████████████████████████████| 1.3 MB 28.0 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [2]:
import mlxtend 
mlxtend.__version__

'0.19.0'

In [3]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [4]:
import pandas as pd
import os

## Dataset import 및 확인

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

basicpath = '/content/drive/MyDrive/Univ/4-1/Data Science/Practice/5주차'

Mounted at /content/drive


In [6]:
path = basicpath
file = 'BreadBasket_DMS.csv'
data = pd.read_csv(os.path.join(path, file), index_col = None)

In [7]:
data.head(10)

,Date,Time,Transaction,Item
0,2016.10.30,9:58:11,1,Bread
1,2016.10.30,10:05:34,2,Scandinavian
2,2016.10.30,10:05:34,2,Scandinavian
3,2016.10.30,10:07:57,3,Hot chocolate
4,2016.10.30,10:07:57,3,Jam
5,2016.10.30,10:07:57,3,Cookies
6,2016.10.30,10:08:41,4,Muffin
7,2016.10.30,10:13:03,5,Coffee
8,2016.10.30,10:13:03,5,Pastry
9,2016.10.30,10:13:03,5,Bread


## Data cleaning

### 편의를 위해 소문자로 변환

In [8]:
data['Item'] = data['Item'].str.lower()
data.head(3)

,Date,Time,Transaction,Item
0,2016.10.30,9:58:11,1,bread
1,2016.10.30,10:05:34,2,scandinavian
2,2016.10.30,10:05:34,2,scandinavian


### Data의 none value 제거

In [9]:
data.isnull().sum()

Date           0
Time           0
Transaction    0
Item           0
dtype: int64

In [10]:
data.iloc[25:30]

,Date,Time,Transaction,Item
25,2016.10.30,10:27:21,11,bread
26,2016.10.30,10:27:21,11,none
27,2016.10.30,10:30:14,12,jam
28,2016.10.30,10:30:14,12,coffee
29,2016.10.30,10:30:14,12,tartine


In [12]:
data.iloc[26]

Date           2016.10.30
Time             10:27:21
Transaction            11
Item                 none
Name: 26, dtype: object

In [13]:
(data['Item'] == 'none').value_counts()

False    20507
True       786
Name: Item, dtype: int64

In [21]:
len(data)

21293

In [16]:
data[data['Item'] == 'none'].index

Int64Index([   26,    38,    39,    66,    80,    85,   126,   140,   149,
              167,
            ...
            20919, 20964, 21010, 21077, 21080, 21108, 21122, 21254, 21255,
            21266],
           dtype='int64', length=786)

In [21]:
data = data.drop(data[data['Item'] == 'none'].index)

In [22]:
len(data)

20507

In [23]:
len(data['Item'].unique())

94

## Data 간단 분석_item당 등장횟수

In [24]:
top10_items = data['Item'].value_counts().head(10)
top10_items

coffee           5471
bread            3325
tea              1435
cake             1025
pastry            856
sandwich          771
medialuna         616
hot chocolate     590
cookies           540
brownie           379
Name: Item, dtype: int64

## FP tree 활용

### Data형태 변환: Vertical format

In [25]:
hot_encoded_data = data.groupby(['Transaction','Item'])['Item'].count().unstack().fillna(0)
# unstack 은 옆으로 쭉 펼친거에요

# 어떤 데이터를 vector로 만드는 것이 label encoding
'''
a = [1,0,0]
b = [0,1,0]
c = [0,0,1]
이런식으로 하나의 hot한 1을 가지는 encoding으로 하는거라 one hot encoding이라고 한다

하나의 transaction을 벡터 하나로 encoding 하고 싶은거에요
걔네들을 hot encoding 으로. 여러개의 1과 0을 갖는 형태로 encoding을 원하는거에요

http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/
'''

'\na = [1,0,0]\nb = [0,1,0]\nc = [0,0,1]\n이런식으로 하나의 hot한 1을 가지는 encoding으로 하는거라 one hot encoding이라고 한다\n\n하나의 transaction을 벡터 하나로 encoding 하고 싶은거에요\n걔네들을 hot encoding 으로. 여러개의 1과 0을 갖는 형태로 encoding을 원하는거에요\n\nhttp://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/\n'

In [26]:
hot_encoded_data.head(9)

Item,adjustment,afternoon with the baker,alfajores,argentina night,art tray,bacon,baguette,bakewell,bare popcorn,basket,...,the bart,the nomad,tiffin,toast,truffles,tshirt,valentine's card,vegan feast,vegan mincepie,victorian sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
#하나의 transaction에 2개 이상의 동일한 item이 들어간 경우 1개로 취급. 
#item 간의 관계가 중요. 동일한 item끼리는 중요하지 않음
def encode_units(x): 
    if x <= 0:
        return 0
    if x >= 1:
        return 1

hot_encoded_data = hot_encoded_data.applymap(encode_units)

### FP tree

In [28]:
frequent_itemsets = fpgrowth(hot_encoded_data, min_support=0.01, use_colnames=True)

# use column name 기본적으로는 False에요 data mining할때 String형태의 데이터로 쓰지 않고, int형태로 바꿔서 쓰게 된다
# 내부적으로 0부터 시작하는 데이터로 column바꿔서 사용합니다 그래서 use_colnames하면 그걸 다시 이름으로 바꿔서 해줘

In [29]:
frequent_itemsets

,support,itemsets
0,0.327205,(bread)
1,0.029054,(scandinavian)
2,0.058320,(hot chocolate)
3,0.054411,(cookies)
4,0.015003,(jam)
...,...,...
56,0.019651,"(brownie, coffee)"
57,0.010777,"(brownie, bread)"
58,0.023666,"(toast, coffee)"
59,0.018067,"(scone, coffee)"


In [30]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.4)
# top 10에서는 안나왔지만 toast coffee가 같이 나가더라 이런게 knowledge인거지

In [31]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(hot chocolate),(cake),0.058320,0.103856,0.011410,0.195652,1.883874,0.005354,1.114125
1,(cake),(hot chocolate),0.103856,0.058320,0.011410,0.109868,1.883874,0.005354,1.057910
2,"(bread, coffee)",(pastry),0.090016,0.086107,0.011199,0.124413,1.444872,0.003448,1.043749
3,(pastry),"(bread, coffee)",0.086107,0.090016,0.011199,0.130061,1.444872,0.003448,1.046033
4,(tea),(cake),0.142631,0.103856,0.023772,0.166667,1.604781,0.008959,1.075372
5,(cake),(tea),0.103856,0.142631,0.023772,0.228891,1.604781,0.008959,1.111865
6,"(tea, coffee)",(cake),0.049868,0.103856,0.010037,0.201271,1.937977,0.004858,1.121962
7,(cake),"(tea, coffee)",0.103856,0.049868,0.010037,0.096643,1.937977,0.004858,1.051779
8,(tea),(sandwich),0.142631,0.071844,0.014369,0.100741,1.402222,0.004122,1.032134
9,(sandwich),(tea),0.071844,0.142631,0.014369,0.200000,1.402222,0.004122,1.071712


In [32]:
rules[rules['confidence'] > 0.17]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(hot chocolate),(cake),0.058320,0.103856,0.011410,0.195652,1.883874,0.005354,1.114125
5,(cake),(tea),0.103856,0.142631,0.023772,0.228891,1.604781,0.008959,1.111865
6,"(tea, coffee)",(cake),0.049868,0.103856,0.010037,0.201271,1.937977,0.004858,1.121962
9,(sandwich),(tea),0.071844,0.142631,0.014369,0.200000,1.402222,0.004122,1.071712
10,(toast),(coffee),0.033597,0.478394,0.023666,0.704403,1.472431,0.007593,1.764582


## Apriori 알고리즘과의 성능 비교

In [34]:
from mlxtend.frequent_patterns import apriori

In [ ]:
%timeit -n 100 -r 10 frequent_itemsets_apriori = apriori(hot_encoded_data, min_support=0.005, use_colnames=True)

In [ ]:
%timeit -n 100 -r 10 frequent_itemsets = fpgrowth(hot_encoded_data, min_support=0.005, use_colnames=True)